# Emotions Recognition AI Project

### 사용할 모델 다운로드
### fece detection adas 0001 다운로드

In [1]:
! omz_downloader --name face-detection-adas-0001 --precision FP16

################|| Downloading face-detection-adas-0001 ||################

========== Downloading C:\BrainAI\Day5-emotion\intel\face-detection-adas-0001\FP16\face-detection-adas-0001.xml
... 100%, 304 KB, 295 KB/s, 1 seconds passed

========== Downloading C:\BrainAI\Day5-emotion\intel\face-detection-adas-0001\FP16\face-detection-adas-0001.bin
... 49%, 1024 KB, 169 KB/s, 6 seconds passed
... 99%, 2048 KB, 263 KB/s, 7 seconds passed
... 100%, 2056 KB, 264 KB/s, 7 seconds passed



### emotions-recognition-retail-0003 다운로드

In [3]:
! omz_downloader --name emotions-recognition-retail-0003 --precision FP16

################|| Downloading emotions-recognition-retail-0003 ||################

========== Downloading C:\BrainAI\Day5-emotion\intel\emotions-recognition-retail-0003\FP16\emotions-recognition-retail-0003.xml
... 100%, 54 KB, 55 KB/s, 0 seconds passed

========== Downloading C:\BrainAI\Day5-emotion\intel\emotions-recognition-retail-0003\FP16\emotions-recognition-retail-0003.bin
... 21%, 1024 KB, 590 KB/s, 1 seconds passed
... 42%, 2048 KB, 665 KB/s, 3 seconds passed
... 63%, 3072 KB, 796 KB/s, 3 seconds passed
... 84%, 4096 KB, 897 KB/s, 4 seconds passed
... 100%, 4848 KB, 969 KB/s, 5 seconds passed



## 필요한 라이브러리 임포트

In [4]:
import openvino as ov
import cv2
import numpy as np

import matplotlib.pyplot as plt
from pathlib import Path

### Inference 할 디바이스 확인

In [5]:
core = ov.Core()
options=core.available_devices

options

['CPU', 'GPU', 'NPU']

### Load the Model

In [8]:
core=ov.Core()
# 얼굴인식모델 먼저 준비
model=core.read_model(model="./models/face-detection-adas-0001.xml")
face_model=core.compile_model(model=model, device_name="CPU")

face_input_layer=face_model.input(0)
face_output_layer=face_model.output(0)

print("Input layer shape:", face_input_layer.shape)
print("Onput layer shape:", face_output_layer.shape)

Input layer shape: [1,3,384,672]
Onput layer shape: [1,1,200,7]


In [10]:
model=core.read_model(model="./models/emotions-recognition-retail-0003.xml")
emotion_model=core.compile_model(model=model, device_name="CPU")

emotion_input_layer=emotion_model.input(0)
emotion_output_layer=emotion_model.output(0)

print("Input layer shape:", emotion_input_layer.shape)
print("Onput layer shape:", emotion_output_layer.shape)

Input layer shape: [1,3,64,64]
Onput layer shape: [1,5,1,1]


### Load Image

In [32]:
frame = cv2.imread("images/emotion.jpg")
resized_frame = cv2.resize(src=frame, dsize=(672, 384)) 
transposed_frame = resized_frame.transpose(2, 0, 1)
input_frame = np.expand_dims(transposed_frame, 0)

### DrauBoundingBoxes() 함수만들기

In [31]:
def DrawBoundingBoxes(output, frame, conf=0.5):
    
    boxes = []   # 얼굴 위치를 저장할 변수
    
    canvas = frame.copy()
    h,w,_ = canvas.shape 

    predictions = output[0][0]            # 하위 집합 데이터 프레임
    confidence = predictions[:,2]         # conf 값 가져오기 [image_id, label, conf, x_min, y_min, x_max, y_max]

    top_predictions = predictions[(confidence>conf)]         # 임계값보다 큰 conf 값을 가진 예측만 선택

    for detection in top_predictions:
        box = (detection[3:7] * np.array([w, h, w, h])).astype("int") # 상자 위치 결정
        (xmin, ymin, xmax, ymax) = box   # xmin, ymin, xmax, ymax에 상자 위치 값 지정
        cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), (0, 0, 255), 2) # 사각형 그리기

        
        
        boxes.append(box)     #이미지에 박스를 그린 얼굴의 위치 저장
   
    return boxes

### 감정을 예측하여 이미지에 예측값 출력 함수 작성

In [24]:
def DrawText(output, frame, face_position):
    # emotions 딕셔너리 생성
    emotions = {
        0:"neutral",
        1:"happy",
        2:"sad",
        3:"surprise",
        4:"anger"
    }
    # 딕셔너리 출력하기
    #for key, value in emotions.items():
    #    print(key, value, end='      ')
    #print()
        
    predictions = output[0,:,0,0]              # 5개의 감정 예측값 저장
    print("predictions : " + str(predictions))
    
    topresult_index = np.argmax(predictions)   # 5개의 감정 예측값 중 가장 높은값의 위치 저장
    #print("topresult_index : " + str(topresult_index))
    
    emotion = emotions[topresult_index]        # emotions에서 topresult_index 값에 해당하는 감정 저장
    #print("emotion : " + emotion)
    
    cv2.putText(frame, emotion,                 # 예측한 감정값 이미지에 출력하기
                (face_position[0],face_position[1]),    #xmin, ymin 값을 가져와 위치 설정
                cv2.FONT_HERSHEY_SIMPLEX, 1, 
                (255, 0,0), 2)

### 얼굴 찾고 박스그려라

In [25]:
face_output = face_model([input_frame])[face_output_layer]  # 얼굴찾고
boxes = DrawBoundingBoxes(face_output, frame, conf=0.5)     # 박스 그리고

In [26]:
if boxes is not None:
    
    for box in boxes:          #boxes에 저장된 얼굴의 위치들을 하나씩 box에 전달
    
        xmin, ymin, xmax, ymax = box      #box에 저장된 좌표 저장
        emotion_input = frame[ymin:ymax,xmin:xmax]         #이미지에서 해당 얼굴 위치를 찾아 저장
        
        # 감정 인식 모델을 사용하기 위해 이미지 전처리
        # Input layer shape:  [1,3,64,64]
        resized_image = cv2.resize(src=emotion_input, dsize=(64, 64))      #이미지 사이즈 변경  (64,64,3)
        transposed_image = resized_image.transpose(2, 0, 1)                #shape 위치 변경    (3,64,64)
        input_image = np.expand_dims(transposed_image, 0)                  #차원 확장 (1,3,64,64)

        emotion_output = emotion_model([input_image])[emotion_output_layer]  # 감정 추론
        DrawText(emotion_output, frame, box)   # 추론의 결과값 이미지에 출력하기

predictions : [0.7655954  0.08088928 0.0355671  0.0031756  0.1147726 ]


In [27]:
cv2.imshow("emotion-recognition", frame)

cv2.waitKey(0)
cv2.destroyAllWindows()

## 백그라운드+감정 출력

In [29]:
def AddBackground(frame, bg):

    frame_h, frame_w = frame.shape[0], frame.shape[1]
    new_h = 500
    new_w = int((new_h/frame_h)*frame_w)
    frame_resize = cv2.resize(frame, (new_w, new_h))

    xmax = bg.shape[1] - 350  # x좌표위치를 설정함.
    ymax = bg.shape[0] - 175  # y좌표위치를 설정함.
    xmin = xmax - new_w
    ymin = ymax - new_h

    bg[ymin:ymax, xmin:xmax] = frame_resize

    return bg

background = "./images/Background2.jpg"  #사용할 배경화면 경로
#canvas = DrawBoundingBoxes(face_output, frame, conf=0.5)  
bg = cv2.imread(background)

deployment = AddBackground(frame, bg)
cv2.imshow("Deployment", deployment)

cv2.waitKey(0)
cv2.destroyAllWindows()

## Webcam

In [33]:
camera = cv2.VideoCapture(0) #create a VideoCapture object with the 'first' camera (your webcam)
background = "./images/Background2.jpg"
bg = cv2.imread(background)

while(True):
    ret, frame = camera.read()             # Capture frame by frame      
    if ret == False:
        break
    
    resized_frame = cv2.resize(src=frame, dsize=(672, 384)) 
    transposed_frame = resized_frame.transpose(2, 0, 1)
    input_frame = np.expand_dims(transposed_frame, 0)    
    
    face_output = face_model([input_frame])[face_output_layer]
## ***************************************************************** 이부분추가    
    boxes = DrawBoundingBoxes(face_output, frame, conf=0.5)
    
    if boxes is not None:
    
        for box in boxes:          #boxes에 저장된 얼굴의 위치들을 하나씩 box에 전달
    
            xmin, ymin, xmax, ymax = box      #box에 저장된 좌표 저장
            emotion_input = frame[ymin:ymax,xmin:xmax]         #이미지에서 해당 얼굴 위치를 찾아 저장
        
            # 감정 인식 모델을 사용하기 위해 이미지 전처리
            # Input layer shape:  [1,3,64,64]
            resized_image = cv2.resize(src=emotion_input, dsize=(64, 64))      #이미지 사이즈 변경  (64,64,3)
            transposed_image = resized_image.transpose(2, 0, 1)                #shape 위치 변경    (3,64,64)
            input_image = np.expand_dims(transposed_image, 0)                  #차원 확장 (1,3,64,64)

            emotion_output = emotion_model([input_image])[emotion_output_layer]  # 감정 추론
            DrawText(emotion_output, frame, box)   # 추론의 결과값 이미지에 출력하기
## **************************************************************************************    
    deployment = AddBackground(frame, bg)
    
    cv2.imshow('Press Spacebar to Exit', deployment)

    if cv2.waitKey(1) & 0xFF == ord(' '):  # Stop if spacebar is detected
        break

camera.release()                           # Cleanup after spacebar is detected.
cv2.destroyAllWindows()

predictions : [0.08978651 0.03106922 0.04703624 0.777479   0.05462904]
predictions : [0.12041687 0.07321367 0.09640618 0.62984943 0.08011384]
predictions : [0.15836556 0.07256188 0.12813526 0.55345327 0.08748402]
predictions : [0.15537365 0.03464397 0.13473184 0.56625885 0.10899173]
predictions : [0.19978842 0.04416333 0.10412195 0.5159237  0.13600263]
predictions : [0.14844021 0.04911372 0.06932092 0.5830305  0.15009458]
predictions : [0.16192664 0.02964349 0.05586612 0.62254727 0.13001637]
predictions : [0.13151081 0.0151095  0.03656738 0.757192   0.05962027]
predictions : [0.12718959 0.01768103 0.03570569 0.75352216 0.06590157]
predictions : [0.1475747  0.02005397 0.05558413 0.69899815 0.07778911]
predictions : [0.16739354 0.02064403 0.05379976 0.68567735 0.07248535]
predictions : [0.1405746  0.01865188 0.04632727 0.72912794 0.06531829]
predictions : [0.15231824 0.01825512 0.05390104 0.70857954 0.06694599]
predictions : [0.12173483 0.01513059 0.04030327 0.7555735  0.06725791]
predic